# Speech-to-Text Benchmarking using Word Error Rate (WER)

This notebook benchmarks three leading Speech-to-Text engines:
1. OpenAI Whisper  
2. faster-whisper  
3. Vosk  

We evaluate them using **Word Error Rate (WER)** on a small test set of audio files.

## What is Word Error Rate (WER)?

WER is a standard metric for evaluating speech recognition systems.

WER = (Substitutions + Deletions + Insertions) / Number of words in reference

Lower WER = better performance.

In [ ]:
%pip install openai-whisper faster-whisper vosk jiwer soundfile librosa

## Dataset

We use 5 short audio samples (25 - 40 sec) with known ground-truth transcripts.
Each audio file has a corresponding `.txt` file containing the correct transcription.

Directory structure:

/dataset
1.     /audio-1.wav
2.     /audio-2.wav
3.     /audio-3.wav
4.     /audio-4.wav
5.     /audio-5.wav

## Cell 2: Imports

In [ ]:
import os
import whisper
from faster_whisper import WhisperModel
from vosk import Model, KaldiRecognizer
import json
import wave
import soundfile as sf
from jiwer import wer

## Cell 3: Loading Test Audio

In [ ]:
DATASET_PATH = "/dataset"

def load_test_data(dataset_path):
    data = []
    for file in os.listdir(dataset_path):
        if file.endswith(".wav"):
            audio_path = os.path.join(dataset_path, file)
            txt_path = audio_path.replace(".wav", ".txt")
            if os.path.exists(txt_path):
                with open(txt_path, "r", encoding="utf-8") as f:
                    reference = f.read().strip()
                data.append((audio_path, reference))
    return data

test_data = load_test_data(DATASET_PATH)

print(f"Loaded {len(test_data)} audio samples")

## Model 1: OpenAI Whisper
High accuracy multilingual transformer-based model.

In [ ]:
whisper_model = whisper.load_model("base")

## Cell 4: Whisper Transcription + WER

In [ ]:
def evaluate_whisper(model, test_data):
    wers = []
    for audio_path, reference in test_data:
        result = model.transcribe(audio_path)
        prediction = result["text"].strip()
        error = wer(reference.lower(), prediction.lower())
        wers.append(error)
        print(f"Whisper WER for {os.path.basename(audio_path)}: {error}")
    return sum(wers) / len(wers)

whisper_avg_wer = evaluate_whisper(whisper_model, test_data)
print(f"\nAverage Whisper WER: {whisper_avg_wer}")

## Model 2: faster-whisper
Optimized implementation of Whisper using CTranslate2 for faster inference.

In [ ]:
faster_model = WhisperModel("base", device="cpu", compute_type="int8")

## Cell 5: Faster-whisper Transcription + WER

In [ ]:
def evaluate_faster_whisper(model, test_data):
    wers = []
    for audio_path, reference in test_data:
        segments, info = model.transcribe(audio_path)
        prediction = " ".join([segment.text for segment in segments]).strip()
        error = wer(reference.lower(), prediction.lower())
        wers.append(error)
        print(f"faster-whisper WER for {os.path.basename(audio_path)}: {error}")
    return sum(wers) / len(wers)

faster_whisper_avg_wer = evaluate_faster_whisper(faster_model, test_data)
print(f"\nAverage faster-whisper WER: {faster_whisper_avg_wer}")

## Model 3: Vosk
Lightweight offline speech recognition engine.

In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip vosk-model-small-en-us-0.15.zip

In [ ]:
vosk_model = Model("vosk-model-small-en-us-0.15")

## Cell 6:  Vosk Transcription

In [ ]:
def transcribe_vosk(audio_path, model):
    wf = wave.open(audio_path, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)

    result_text = ""

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            result_text += " " + res.get("text", "")

    final_res = json.loads(rec.FinalResult())
    result_text += " " + final_res.get("text", "")

    return result_text.strip()

## Cell 7: Vosk Transcription + WER

In [ ]:
def evaluate_vosk(model, test_data):
    wers = []
    for audio_path, reference in test_data:
        prediction = transcribe_vosk(audio_path, model)
        error = wer(reference.lower(), prediction.lower())
        wers.append(error)
        print(f"Vosk WER for {os.path.basename(audio_path)}: {error}")
    return sum(wers) / len(wers)

vosk_avg_wer = evaluate_vosk(vosk_model, test_data)
print(f"\nAverage Vosk WER: {vosk_avg_wer}")

## Final WER Comparison
Lower is better.

In [ ]:
import pandas as pd

results = {
    "Model": ["Whisper", "faster-whisper", "Vosk"],
    "Average WER": [whisper_avg_wer, faster_whisper_avg_wer, vosk_avg_wer]
}

df = pd.DataFrame(results)
df